In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from GameDayFunctions.fangraphs_projection_2020 import Projection
#from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [755]:
class Draft:

    def __init__(self, projections_object, 
                 draft_position = 6,
                 number_teams = 12,
                 roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5},
                 batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'],
                 pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] ):

        self.number_teams = number_teams
        self.number_rounds = sum(roster_spots.values())
        self.draft_position = draft_position - 1 # e.g., 1st pick is 0!
        self.round_number = 0
        self.draft_number = 0
        self.player_projections = projections_object
        self.remaining_ranked_players = projections_object.all_rank 
        self.roto_stats_batting = pd.DataFrame(columns =  batter_stats[1:])
        self.roto_stats_pitching = pd.DataFrame(columns =  pitcher_stats[1:])
        self.teams = {}
        for i in np.arange(number_teams):
            roto_stats = {}
            roto_stats['batting_stats'] = pd.DataFrame(columns =  batter_stats)
            roto_stats['pitching_stats'] = pd.DataFrame(columns =  pitcher_stats)
            roto_stats['roster_spots'] = roster_spots.copy()
            roto_stats['roster'] = {}
            self.teams[i] = roto_stats
    
    # Find Resulting Standings
    def tabulate_roto(self, teams):
        #pdb.set_trace()
        roto_stats_batting = self.roto_stats_batting.copy()
        roto_stats_pitching = self.roto_stats_pitching.copy()
        for iteam in np.arange(self.number_teams):
            raw_team_batting = teams[iteam]['batting_stats']
            raw_team_pitching = teams[iteam]['pitching_stats']
            roto_team_batting = raw_team_batting.mean()
            roto_team_pitching = raw_team_pitching.mean()
            if 'AVG' in raw_team_batting:
                roto_team_batting['AVG'] = (raw_team_batting['AVG']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum()
            if 'OPS' in raw_team_batting:
                roto_team_batting['OPS'] = (raw_team_batting['OPS']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum() 
            if 'ERA' in raw_team_pitching:
                roto_team_batting['ERA'] = (raw_team_pitching['ERA']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum() 
            if 'WHIP' in raw_team_pitching:
                roto_team_batting['WHIP'] = (raw_team_pitching['WHIP']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum()
            batting_stat_names = self.roto_stats_batting.columns.values.tolist()
            pitching_stat_names = self.roto_stats_pitching.columns.values.tolist()
            roto_stats_batting = roto_stats_batting.append(roto_team_batting[batting_stat_names],ignore_index = True)
            roto_stats_pitching = roto_stats_pitching.append(roto_team_pitching[pitching_stat_names],ignore_index = True)
            
        roto_team_stats = pd.concat([roto_stats_batting, roto_stats_pitching], axis=1, sort=False)
        roto_standings_avgs = pd.concat([roto_stats_batting.rank(ascending=False), roto_stats_pitching.rank(ascending=False).rename(columns={"BB": "BBP"})], axis=1, sort=False)
        roto_standings_cnts = pd.concat([roto_stats_batting.rank(), roto_stats_pitching.rank().rename(columns={"BB": "BBP"})], axis=1, sort=False)
        avg_stats = ['CS','BBP','ERA','WHIP','BSV']
        for avg_stat in avg_stats:
            if avg_stat in roto_standings_avgs:
                roto_standings_cnts[avg_stat] = roto_standings_avgs[avg_stat]
        roto_standings = roto_standings_cnts.sum(axis=1).sort_values(ascending=False)
        roto_placement = roto_standings.index.get_loc(self.draft_position)
        #pdb.set_trace()
        return roto_stats_batting, roto_stats_pitching, roto_standings, roto_placement
    
    
    # Do the entire draft one round at a time
    def draft_all(self):
        for iround in np.arange(self.number_rounds):
            self.draft_round(iround)
            self.round_number += 1
            #pdb.set_trace()
        self.roto_stats_batting, self.roto_stats_pitching, self.roto_standings, self.roto_placement = self.tabulate_roto(self.teams)
      
    # Draft each round one team at a time.  When reaching "draft_position", stop and to pseudo_drafts to figure out best choice. 
    def draft_round(self, round_key):
        
        # Reverse draft order every other round
        draft_order = np.arange(self.number_teams)
        if round_key % 2 == 1:
            draft_order = draft_order[::-1]
        
        # Reset draft_number at start of round
        self.draft_number = self.number_teams - len(draft_order)
        
        # Draft each round one team at a time
        for iteam in draft_order:
            
            # When team is draft_position, search for best pick.
            if iteam == self.draft_position:
                best_pick = self.find_best_pick(iteam, round_key)
                pdb.set_trace()
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam, force_pick = best_pick)
            else: 
                self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            #self.teams, self.remaining_ranked_players = self.draft_next_best(iteam)
            self.draft_number += 1
            
        #pdb.set_trace()

        
    def draft_remaining(self, teams, df, draft_round):
        
        teams_copy = teams.copy()
        df_copy = df.copy()
        
        #pdb.set_trace()
        for iround in range(draft_round,self.number_rounds):
            
            draft_order = np.arange(self.number_teams)
            
            # Reverse draft order every other round
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
            
            # For the very first round begin at beginning self.draft_position
            if iround == draft_round:
                if iround % 2 == 1:
                    starting_position = self.number_teams - self.draft_position;
                else:
                    starting_position = self.draft_position + 1;
                draft_order = draft_order[starting_position:] 
                
            for iteam in draft_order:
                teams_copy, df_copy = self.draft_next_best_debug(iteam, teams = teams_copy, df = df_copy)

        #pdb.set_trace()
        return teams_copy, df_copy
        
    def find_best_pick(self, team_key, round_key):
        # find_best_pick returns iloc, the index (of df) of the optimal pick
        
        df = self.remaining_ranked_players
        all_unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
        # Figure out if only BN is left, in which case all options are open
        if ('BN' in all_unfilled_positions) and (len(all_unfilled_positions) == 1):
            unfilled_positions = ['C', '1B', '2B', '3B', 'SS', 'OF','Util','SP','RP']
        else: 
            # If UTIL is unfilled, all fielding positions are options.  
            if self.teams[team_key]['roster_spots']['UTIL'] > 0:
                #ups = 'C|1B|2B|3B|SS|OF'
                unfilled_positions = ['C', '1B', '2B', '3B', 'SS', 'OF']
                if self.teams[team_key]['roster_spots']['SP'] > 0:
                    #ups = ups+'|SP'
                    unfilled_positions.append('SP')
                if self.teams[team_key]['roster_spots']['RP'] > 0:
                    #ups = ups+'|RP'
                    unfilled_positions.append('RP')
                #unfilled_positions = ups.split('|')
            else: 
                unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]
                #ups = '|'.join(unfilled_positions)
        ups = '|'.join(unfilled_positions)
        print(ups)  
        
        # Begin with next best player.        
        idx_eligible = [0]
        
        # Find index of best player at each remaining position
        for iunfilled in unfilled_positions:
            if (iunfilled != 'UTIL') & (iunfilled != 'BN'):
                idx_position = [i for i, val in enumerate(df.EligiblePosition.str.contains(iunfilled)) if val] 
                filled = False
                jdx = 0
                while filled == False:
                    if idx_position[jdx] in idx_eligible:
                        jdx+=1
                    else:
                        idx_eligible.append(idx_position[jdx])
                        filled = True
        
        # Get rid of doubles (1B and OF is particularly prone)
        idx_eligible = np.unique(idx_eligible)
        print('Picking from:')
        print(df.iloc[idx_eligible])
        
        #################################
        # START OF LOOP TO FIND BEST PLAYER
        player_based_draft_outcomes = {}

        # Loop over eligible players, then finish the draft
        for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):
            
            # make a copy of teams to finish drafting
            teams_copy = self.teams.copy()
            df_copy = df.copy()
            iplayer = df_copy.iloc[iposition].PLAYER
            #idx_player = df_copy.iloc[iposition].index

            # Draft looping through idx_eligible
            df_copy,drafted_player=df_copy.drop(df_copy.iloc[iposition:iposition+1].index),df_copy.iloc[iposition:iposition+1]
            position = unfilled_positions[icounter]
            #pdb.set_trace()
            # Already know there are open positions, so is this necessary?
            #eligible_positions = drafted_player.EligiblePosition.values[0] 
            #position = self.get_optimal_position(eligible_positions, teams_copy[team_key]['roster_spots'])
            
            # Check if Position is a Pitcher or Batter
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                teams_copy[team_key]['pitching_stats'] = teams_copy[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                teams_copy[team_key]['batting_stats'] = teams_copy[team_key]['batting_stats'].append(statline[0:1])

            # LOOP OVER WHOLE REST OF THE DRAFT HERE...
            teams_loop, df_loop = self.draft_remaining(teams_copy, df_copy, round_key)
            
            # Calculate the best pseudo-standings
            #pdb.set_trace()
            pseudo_batting_stats, pseudo_pitching_stats, pseudo_standings, pseudo_placement = self.tabulate_roto(teams_loop)
            
            # Decide best choice
            # LOGIC...
            
            # Store the result.  
            player_based_draft_outcomes = {iplayer:pseudo_placement}
            
            pdb.set_trace()
        
        # Decide on best choice and return 
        best_pick = 5
        
        return best_pick
        # END OF LOOP TO FIND BEST PLAYER
        #################################

    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
        
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while (pick_ok == False) & (len([k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]) > 0):
                df_copy = df.copy()
                pick_ok = True

                # Draft next in list.  Do this way (idf) so that if you don't take player, they are not removed for next picker
                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                
                # Get eligible positions
                eligible_positions = drafted_player.EligiblePosition.values[0]
                
                # Find best position opening.  Return 0 if no room
                position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                
                # Update Rosters with drafted_player, Loop otherwise
                if position == 0:
                    unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]           
                    #print('Not Drafting '+eligible_positions+" "+drafted_player.PLAYER+' for '+ "/".join(unfilled_positions))
                    idf += 1
                    pick_ok = False
                else:
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)                    
                    df = df_copy                            
                    pick_ok = True
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
        else:
            pdb.set_trace()
            pick = force_pick -1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' force BEST pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best_debug(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
        
        #pdb.set_trace()
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            #while (pick_ok == False) & (len([k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]) > 0):
            while (pick_ok == False):
                df_copy = df.copy()
                pick_ok = True

                # Draft next in list.  Do this way (idf) so that if you don't take player, they are not removed for next picker
                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                
                #pdb.set_trace()
                #print(drafted_player)
                #print(drafted_player.EligiblePosition.values[0])
                if drafted_player.iloc[0].PLAYER == 'Yonder Alonso':
                    pdb.set_trace()
                # Get eligible positions
                eligible_positions = drafted_player.EligiblePosition.values[0]
                
                # Find best position opening.  Return 0 if no room
                position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                
                # Update Rosters with drafted_player, Loop otherwise
                if position == 0:
                    unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]           
                    #print('Not Drafting '+eligible_positions+" "+drafted_player.PLAYER+' for '+ "/".join(unfilled_positions))
                    idf += 1
                    pick_ok = False
                else:
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)                    
                    df = df_copy                            
                    pick_ok = True
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
        else:
            pdb.set_trace()
            pick_ok = True
            pick = force_pick -1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' force BEST pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        #if pick_ok == True:
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]

        return teams, df
    
    def get_optimal_position(self, positions_in, roster_spots):
        # Return in the order of most valuable position to fill
        single_positions = positions_in.split('/')
        # Check Pitchers
        P = False
        if ('RP' in single_positions):
            P = True
            if (roster_spots['RP'] > 0):
                return  'RP' 
        elif ('SP' in single_positions):
            P = True
            if (roster_spots['SP'] > 0):
                return  'SP'
        if P == True:
            if (roster_spots['P'] > 0):
                return  'P'
            elif (roster_spots['BN'] > 0):
                return 'BN'
            else:
                return 0
        # Check Hitters
        Util = False
        if ('C' in single_positions):
            Util = True
            if (roster_spots['C'] > 0):
                return  'C'
        elif ('2B' in single_positions): 
            Util = True
            if (roster_spots['2B'] > 0):
                return  '2B'
        elif ('SS' in single_positions):
            Util = True
            if (roster_spots['SS'] > 0):
                return  'SS'
        elif ('OF' in single_positions):
            Util = True
            if (roster_spots['OF'] > 0):
                return  'OF'
        elif ('3B' in single_positions):
            Util = True
            if (roster_spots['3B'] > 0):
                return  '3B'
        elif ('1B' in single_positions):
            Util = True 
            if (roster_spots['1B'] > 0):
                return  '1B'
        elif ('Util' in single_positions):
            Util = True 
            if (roster_spots['UTIL'] > 0):
                return  'UTIL'                
        if Util == True:
            if (roster_spots['UTIL'] > 0):
                return  'UTIL'
            elif (roster_spots['BN'] > 0):
                return 'BN'
            else:
                return 0
        else:
            return 0

In [756]:
zips = Draft(player_ranking_zips, number_teams = 12)
steam = Draft(player_ranking_steam, number_teams = 12)

In [757]:
zips.draft_all()

C|1B|2B|3B|SS|OF|SP|RP
Picking from:
                PLAYER EligiblePosition
Rank                                   
6       Anthony Rendon               3B
7       Cody Bellinger            1B/OF
9         Alex Bregman            3B/SS
11    Justin Verlander               SP
12       J.D. Martinez               OF
14    Francisco Lindor               SS
15     Whit Merrifield            2B/OF
27       Roberto Osuna               RP
185      J.T. Realmuto                C
> <ipython-input-755-42e30ff1c849>(218)find_best_pick()
-> player_based_draft_outcomes = {iplayer:pseudo_placement}


(Pdb)  cont


> <ipython-input-755-42e30ff1c849>(179)find_best_pick()
-> for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):


(Pdb)  player_based_draft_outcomes


{'Anthony Rendon': 8}


...

{'batting_stats':         AB    R   1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
11477  539   99   91  34  4  34  106  24  77  0.302  0.964
13613  576   88  105  33  9  22   78  10  52  0.293  0.850
17350  624  112  102  44  3  33  110   9  48  0.292  0.875
3473   518   82   87  30  3  27  101   6  70  0.284  0.898
9241   541   84  102  28  5  19   76  28  26  0.285  0.792
15149  582   83   95  30  3  31   92   1  51  0.273  0.832
13624  506   78   84  36  2  19   87   2  47  0.279  0.813
15518  598   76  113  28  9  14   64  21  31  0.274  0.734
11609  402   54   60  21  2  19   65   3  46  0.254  0.800, 'pitching_stats':         IP   W  L CG SHO  SV  BB   SO   ERA  WHIP BSV
42   119.7   9  6  1   1   0  48  162  3.53  1.19   0
4    195.7  13  8  1   1   0  34  213  3.63  1.11   0
264   59.0   4  2  0   0  27  17   77  3.05  1.07   2
28   159.0  10  9  1   1   0  37  174  3.68  1.11   0
633   57.3   4  3  0   0   0  17   58  4.08  1.26   0
96   136.0   8  7  1   1   0  30  118  4.37  1.22

(Pdb)  teams[5]


{'batting_stats':         AB    R   1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
12861  525   89   82  35  2  26   97   5  68  0.276  0.862
5417   544   90  110  29  3  24   78  16  48  0.305  0.868
16376  516   83   73  27  1  31   96   6  78  0.256  0.853
15362  519   77   97  32  4  19   71   7  35  0.293  0.835
18363  536   81   90  31  5  16   69  29  38  0.265  0.767
14344  514   80   67  28  0  35   98   1  60  0.253  0.851
9847   420   69   64  20  2  21   64   6  65  0.255  0.820
13769  461   62   71  22  8  14   63  42  23  0.249  0.708
15998  547  108   84  32  4  40  120  16  88  0.293  0.977, 'pitching_stats':          IP   W  L CG SHO SV  BB   SO   ERA  WHIP BSV
186    67.0   4  2  0   0  0  19   78  3.09  1.12   0
257    59.3   5  3  0   0  0  20   95  3.19  1.06   0
527    68.0   4  3  0   0  0  26   73  3.84  1.29   0
350    57.0   4  2  0   0  0  21   58  3.47  1.25   0
680    52.3   3  3  0   0  0  14   40  3.78  1.24   0
7     186.7  11  7  1   1  0  45  197  3.33  1.17

(Pdb)  exit


BdbQuit: 

In [523]:
zips2.draft_all()

In [525]:
zips2.teams[4]['roster']

{'OF': [['Juan Soto', 'Yordan Alvarez'], 'Giancarlo Stanton'],
 'SP': ['Max Scherzer', 'Corey Kluber'],
 'RP': ['Seth Lugo', 'Brandon Workman'],
 '3B': 'Vladimir Guerrero',
 'P': [['Kenley Jansen', 'Ian Kennedy'], 'Craig Kimbrel'],
 'BN': [[[['Chris Martin', 'José Berríos'], 'Charlie Morton'], 'Randy Dobnak'],
  'Seranthony Domínguez'],
 'SS': 'Carlos Correa',
 '2B': 'Jonathan Villar',
 'UTIL': 'Yasiel Puig',
 '1B': 'Eric Hosmer',
 'C': 'Mitch Garver'}

In [526]:
zips.teams[4]['roster']

{'OF': [['Juan Soto', 'Yordan Alvarez'], 'Giancarlo Stanton'],
 'SP': ['Max Scherzer', 'Corey Kluber'],
 'RP': ['Seth Lugo', 'Brandon Workman'],
 '3B': 'Vladimir Guerrero',
 'P': [['Kenley Jansen', 'Ian Kennedy'], 'Craig Kimbrel'],
 'BN': [[[['Chris Martin', 'José Berríos'], 'Charlie Morton'], 'Randy Dobnak'],
  'Seranthony Domínguez'],
 'SS': 'Carlos Correa',
 '2B': 'Jonathan Villar',
 'UTIL': 'Yasiel Puig',
 '1B': 'Eric Hosmer',
 'C': 'Mitch Garver'}

In [7]:
steam.draft_all()

In [8]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [9]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [10]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}

In [ ]:
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best_old(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
                
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while pick_ok == False:
                df_copy = df.copy()
                pick_ok = True
                #print('Round '+str(self.round_number))

                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                eligible_positions = drafted_player.EligiblePosition.values[0]
                

                if eligible_positions == 'Util':
                    open_positions = [teams[team_key]['roster_spots']['UTIL']]
                else:
                    open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
                
                # If there are no SP, RP, or Fielding Positions left, try P, UTIL, or BN
                if np.sum(open_positions) == 0:
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                            
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:

                    df = df_copy                            
                    position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                    #print(drafted_player.PLAYER+' is a '+position)
                    pick_ok = True

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)
                else:
                    #print('Not Drafting '+drafted_player.PLAYER+' for '+position)
                    # Try while loop again after incrementing idf
                    idf += 1

        else:
            #pdb.set_trace()
            pick = force_pick - 1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' FORCED to pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df